In [30]:
import time
import re
start_time = time.time()

import pandas as pd

In [31]:
MMSTA_path = "../Data/Famille Alpine/MMSTA R0.xlsx"

In [32]:
df_mmsta = pd.read_excel(MMSTA_path)
df_mmsta

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status,Follow-Up Material
0,0,MA13,YFG,F000300397,240103778V,7031160.0,2,NaN,1,0,...,NaN,EX,0,0,NaN,L,NaN,0,NaN,NaN
1,*1,MA13,YSFG,S009231594,Circuit 0234 COFLRYB 0.35 G/W,7030635.0,92,NaN,2,0,...,NaN,YX,0,25,NaN,L,PC,1,NaN,NaN
2,**2,MA13,YDM,7158316660,Single Wire Seal Green,NaN,92,NaN,0,0,...,NaN,ZK,0,2000,R,L,PC,1,NaN,NaN
3,**2,MA13,YDM,7195028702,Terminal 025 Male 0.64mm Unsealed,NaN,92,NaN,0,0,...,NaN,ZK,0,60000,L,L,PC,1,NaN,NaN
4,**2,MA13,YDM,7196146002,Terminal RH Female 0.64mm Sealed+Unseale,NaN,92,NaN,0,0,...,NaN,ZK,0,10000,R,L,PC,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24147,*1,MA13,YDM,430AM4530,Textile/Cloth Tape 45mm B,NaN,2,NaN,0,0,...,NaN,ZK,0,540,NaN,L,MM,22397,NaN,NaN
24148,*1,MA13,YDM,430AU2541,PP Tape 25mm GY,NaN,2,NaN,0,0,...,NaN,ZK,0,4500,NaN,L,MM,3937,NaN,NaN
24149,*1,MA13,YDM,430FF1930,PVC/Vinyl Tape 19mm B,NaN,2,NaN,0,0,...,NaN,ZK,0,3168,NaN,L,MM,100082,NaN,NaN
24150,*1,MA13,YDM,430H31930,Textile/Cloth Tape 19mm B,NaN,2,NaN,0,0,...,NaN,ZK,0,3600,NaN,L,MM,37633,NaN,NaN


In [33]:
df_mmsta[df_mmsta['Level'] == '0'].nunique()

Level                      1
Plant                      1
Material Type              1
Material                  11
Material Description      11
Bill of material          11
Plant-sp.matl status       1
Plant-sp.matl status.1     0
BOM status                 1
BOM status.1               1
Change Number              1
Changed on                 0
MRP Controller             1
MRP Controller.1           0
MRP profile                1
MRP profile.1              0
Lot size                   1
Minimum Lot Size           1
Rounding value             1
Spare part indicator       0
Item Category              1
Component unit             0
Component quantity         1
Status                     0
Follow-Up Material         0
dtype: int64

In [34]:
df_mmsta = df_mmsta[['Level', 'Material Type' , 'Material', 'Material Description', 'Component quantity']]

### Add Columns 'PN','SN1','DES1', 'SN2','DES2', 'SN3','DES3'

In [35]:
def add_columns(df):
    global cols_to_add 
    cols_to_add = ['PN','SN1','DES1', 'SN2','DES2', 'SN3','DES3']
    col_index = df.columns.get_loc('Material Description') + 1
    for i, col in enumerate(cols_to_add):
        df.insert(col_index + i, col, '')
    return df
df_mmsta = add_columns(df_mmsta)
df_mmsta

,Level,Material Type,Material,Material Description,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,0,YFG,F000300397,240103778V,,,,,,,,0
1,*1,YSFG,S009231594,Circuit 0234 COFLRYB 0.35 G/W,,,,,,,,1
2,**2,YDM,7158316660,Single Wire Seal Green,,,,,,,,1
3,**2,YDM,7195028702,Terminal 025 Male 0.64mm Unsealed,,,,,,,,1
4,**2,YDM,7196146002,Terminal RH Female 0.64mm Sealed+Unseale,,,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...
24147,*1,YDM,430AM4530,Textile/Cloth Tape 45mm B,,,,,,,,22397
24148,*1,YDM,430AU2541,PP Tape 25mm GY,,,,,,,,3937
24149,*1,YDM,430FF1930,PVC/Vinyl Tape 19mm B,,,,,,,,100082
24150,*1,YDM,430H31930,Textile/Cloth Tape 19mm B,,,,,,,,37633


### Fill PN(Part Number)

In [11]:
def Fill(level : str):
    for idx in range(df_mmsta.shape[0]):
        if df_mmsta.loc[:,'Level'][idx] == level:
            # PN_arr.append(df_mmsta.loc[idx]['Material Description'])
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx]['Material Description']
            idx_last_l = idx
        else :
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx_last_l]['Material Description']
            # PN_arr.append(df_mmsta.loc[idx_last_l]['Material Description'])
    return df_mmsta

In [12]:
df_mmsta = Fill('0')

### Delete YFG And YDM (column = 'Material Type')

In [13]:
def delete_YFG_YDM():
    return df_mmsta[~df_mmsta['Material Type'].isin(['YFG', 'YDM'])]
df_mmsta = delete_YFG_YDM()

### Processing Material Description column (del Cut Tube , Gaft...)

In [ ]:
# def del_vals(mat_desc : str):
#     vals_to_del = ['ctgf', 'gaft', 'cut tube']
#     for v in vals_to_del:
#         if v in mat_desc.lower():
#             return False
#     return True

# proccessed_log = df_mmsta['Material Description'].apply(del_vals)

# df_mmsta = df_mmsta[proccessed_log].reset_index(drop=True)

In [14]:
df_mmsta.reset_index(drop=True, inplace=True)
df_mmsta

,Level,Material Type,Material,Material Description,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,*1,YSFG,S009231594,Circuit 0234 COFLRYB 0.35 G/W,240103778V,,,,,,,1
1,*1,YSFG,S009231595,Circuit 0235 COFLRYB 0.35 P/W,240103778V,,,,,,,1
2,*1,YSFG,S009231596,Circuit 0236 COFLRYB 0.35 L/W,240103778V,,,,,,,1
3,*1,YSFG,S009231597,Circuit 0237 COFLRYB 0.35 BE/W,240103778V,,,,,,,1
4,*1,YSFG,S009231598,Circuit 0238 COFLRYB 0.35 R/W,240103778V,,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7196,*1,YSFG,S008676721,Cut Tube PL8NA 13 B @ 00120,240108299V,,,,,,,2
7197,*1,YSFG,S008821596,Cut Tube GAFT2 6.5 B/ @ 00765,240108299V,,,,,,,1
7198,*1,YSFG,S009139196,Tube PL8NA 25 B L=205,240108299V,,,,,,,1
7199,*1,YSFG,S008625554,Cut Tube FG2128 25 B @ 00075,240108299V,,,,,,,1


In [15]:
df_mmsta['Level'].unique()

array(['*1', '**2', '***3'], dtype=object)

In [16]:
def Fill_with_col(level : str, cols,df):
    idx_last_l = None
    for idx in range(df.shape[0]):
        if df.loc[idx,'Level'] == level:
            df.loc[idx,cols[0]] = df.loc[idx,'Material']
            df.loc[idx,cols[1]] = df.loc[idx,'Material Description']
            idx_last_l = idx
           
            
        else :
            if idx_last_l is not None:
                df.loc[idx,cols[0]] = df.loc[idx_last_l,'Material']
                df.loc[idx,cols[1]] = df.loc[idx_last_l,'Material Description']
                
            else :
                df.loc[idx,cols[0]] = '(blank)'
                df.loc[idx,cols[1]] = '(blank)'
    return df

for level in df_mmsta['Level'].unique().tolist():
    sn = 'SN'+ level[-1]
    des = 'DES' + level[-1]
    df_mmsta = Fill_with_col(level, [sn, des], df_mmsta)
# df_mmsta = Fill_with_col('*1', ['SN1','DES1'], df_mmsta)

# df_mmsta = Fill_with_col('**2', ['SN2','DES2'], df_mmsta)

# df_mmsta = Fill_with_col('***3', ['SN3','DES3'], df_mmsta)

In [17]:
df_mmsta = df_mmsta[['Level' ,'PN','SN1','DES1','SN2','DES2','SN3','DES3', 'Component quantity']]

df_mmsta


,Level,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,*1,240103778V,S009231594,Circuit 0234 COFLRYB 0.35 G/W,(blank),(blank),(blank),(blank),1
1,*1,240103778V,S009231595,Circuit 0235 COFLRYB 0.35 P/W,(blank),(blank),(blank),(blank),1
2,*1,240103778V,S009231596,Circuit 0236 COFLRYB 0.35 L/W,(blank),(blank),(blank),(blank),1
3,*1,240103778V,S009231597,Circuit 0237 COFLRYB 0.35 BE/W,(blank),(blank),(blank),(blank),1
4,*1,240103778V,S009231598,Circuit 0238 COFLRYB 0.35 R/W,(blank),(blank),(blank),(blank),1
...,...,...,...,...,...,...,...,...,...
7196,*1,240108299V,S008676721,Cut Tube PL8NA 13 B @ 00120,S009231826,Circuit 0651 COFLRYB 1.5 G/BR,S008821712,Circuit 0547 COFLRYB 0.35 G/W,2
7197,*1,240108299V,S008821596,Cut Tube GAFT2 6.5 B/ @ 00765,S009231826,Circuit 0651 COFLRYB 1.5 G/BR,S008821712,Circuit 0547 COFLRYB 0.35 G/W,1
7198,*1,240108299V,S009139196,Tube PL8NA 25 B L=205,S009231826,Circuit 0651 COFLRYB 1.5 G/BR,S008821712,Circuit 0547 COFLRYB 0.35 G/W,1
7199,*1,240108299V,S008625554,Cut Tube FG2128 25 B @ 00075,S009231826,Circuit 0651 COFLRYB 1.5 G/BR,S008821712,Circuit 0547 COFLRYB 0.35 G/W,1


In [18]:
df_mmsta_pivot = pd.pivot_table(df_mmsta, values = 'Component quantity', index = ['Level','SN1','DES1','SN2' , 'DES2','SN3', 'DES3'], 
                                columns = 'PN', aggfunc = 'size', fill_value = ' ')
# df_mmsta_pivot = df_mmsta_pivot.reset_index()

df_mmsta_pivot

PN                                                                                                                            240100320V  \
Level SN1        DES1                        SN2        DES2                        SN3        DES3                                        
***3  S009232044 Super Group 002 See drawing S008822118 Twisted Wire 13   ZZ        S008821712 Circuit 0547 COFLRYB 0.35 G/W           1   
                                                                                    S008821713 Circuit 0548 COFLRYB 0.35 GY/W          1   
                                             S008822119 Twisted Wire 12   ZZ        S008821714 Circuit 0545 COFLRYB 0.35 G/W           1   
                                                                                    S008821715 Circuit 0546 COFLRYB 0.35 GY/W          1   
                                             S009231951 Twisted Wire 14   ZZ        S009231652 Circuit 0549 COFLRYB 0.35 GY/W          1   
...                                                                                                                                  ...   
*1    S009232347 Super Group 032 See drawing S009231828 Circuit 0583 A3ZP 0.35 LG/W S008821643 Circuit 0575 COFLRYB 0.35 BR/W              
      S009232348 Super Group 009 See drawing S008822431 Twisted Wire 18   ZZ        S008822214 Circuit 0593 COFLRYB 0.35 L/W               
      S009232350 Super Group 045 See drawing S008822106 Twisted Wire 46   ZZ        S008821679 Circuit 0645 COFLRYB 0.35 L/W               
                                             S008822118 Twisted Wire 13   ZZ        S008821712 Circuit 0547 COFLRYB 0.35 G/W               
      S009232399 Super Group 005 See drawing S008822106 Twisted Wire 46   ZZ        S008821679 Circuit 0645 COFLRYB 0.35 L/W               

PN                                                                                                                            240102289V  \
Level SN1        DES1                        SN2        DES2                        SN3        DES3                                        
***3  S009232044 Super Group 002 See drawing S008822118 Twisted Wire 13   ZZ        S008821712 Circuit 0547 COFLRYB 0.35 G/W           1   
                                                                                    S008821713 Circuit 0548 COFLRYB 0.35 GY/W          1   
                                             S008822119 Twisted Wire 12   ZZ        S008821714 Circuit 0545 COFLRYB 0.35 G/W           1   
                                                                                    S008821715 Circuit 0546 COFLRYB 0.35 GY/W          1   
                                             S009231951 Twisted Wire 14   ZZ        S009231652 Circuit 0549 COFLRYB 0.35 GY/W          1   
...                                                                                                                                  ...   
*1    S009232347 Super Group 032 See drawing S009231828 Circuit 0583 A3ZP 0.35 LG/W S008821643 Circuit 0575 COFLRYB 0.35 BR/W              
      S009232348 Super Group 009 See drawing S008822431 Twisted Wire 18   ZZ        S008822214 Circuit 0593 COFLRYB 0.35 L/W           1   
      S009232350 Super Group 045 See drawing S008822106 Twisted Wire 46   ZZ        S008821679 Circuit 0645 COFLRYB 0.35 L/W               
                                             S008822118 Twisted Wire 13   ZZ        S008821712 Circuit 0547 COFLRYB 0.35 G/W               
      S009232399 Super Group 005 See drawing S008822106 Twisted Wire 46   ZZ        S008821679 Circuit 0645 COFLRYB 0.35 L/W               

PN                                                                                                                            240102365V  \
Level SN1        DES1                        SN2        DES2                        SN3        DES3                                        
***3  S009232044 Super Group 002 See drawing S008822118 Twisted Wire 13   ZZ  

In [23]:
import xlwings as xw

wb = xw.Book()
if len(wb.sheets) == 0:
	sheet_mmsta = wb.sheets.add(name="sep mmsta")
else:
	sheet_mmsta = wb.sheets[0]
	sheet_mmsta.name = "sep mmsta"

In [24]:
df_mmsta_pivot = df_mmsta_pivot.reset_index()

In [25]:
df_mmsta_pivot.loc[df_mmsta_pivot['Level'] == '*1', ['SN2', 'DES2', 'SN3', 'DES3']] = '(blank)'
df_mmsta_pivot.loc[df_mmsta_pivot['Level'] == '**2', ['SN3', 'DES3']] = '(blank)'
df_mmsta_pivot

PN,Level,SN1,DES1,SN2,DES2,SN3,DES3,240100320V,240102289V,240102365V,240103778V,240104621V,240107245V,240107760V,240107922V,240108299V,240108828V,240109272V
0,***3,S009232044,Super Group 002 See drawing,S008822118,Twisted Wire 13 ZZ,S008821712,Circuit 0547 COFLRYB 0.35 G/W,1,1,1,1,1,1,1,1,1,1,1
1,***3,S009232044,Super Group 002 See drawing,S008822118,Twisted Wire 13 ZZ,S008821713,Circuit 0548 COFLRYB 0.35 GY/W,1,1,1,1,1,1,1,1,1,1,1
2,***3,S009232044,Super Group 002 See drawing,S008822119,Twisted Wire 12 ZZ,S008821714,Circuit 0545 COFLRYB 0.35 G/W,1,1,1,1,1,1,1,1,1,1,1
3,***3,S009232044,Super Group 002 See drawing,S008822119,Twisted Wire 12 ZZ,S008821715,Circuit 0546 COFLRYB 0.35 GY/W,1,1,1,1,1,1,1,1,1,1,1
4,***3,S009232044,Super Group 002 See drawing,S009231951,Twisted Wire 14 ZZ,S009231652,Circuit 0549 COFLRYB 0.35 GY/W,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5345,*1,S009232347,Super Group 032 See drawing,(blank),(blank),(blank),(blank),,,,,,,,,1,,
5346,*1,S009232348,Super Group 009 See drawing,(blank),(blank),(blank),(blank),,1,,,,,,,,,
5347,*1,S009232350,Super Group 045 See drawing,(blank),(blank),(blank),(blank),,,,,,,,,,,1
5348,*1,S009232350,Super Group 045 See drawing,(blank),(blank),(blank),(blank),,,,,,,,,,1,


In [26]:
def level_sorter(level):
    return int(level[-1])

df_mmsta_pivot['level_sort'] = df_mmsta_pivot['Level'].apply(level_sorter)
df_mmsta_pivot = df_mmsta_pivot.sort_values('level_sort').drop(columns='level_sort')

df_mmsta_pivot

PN,Level,SN1,DES1,SN2,DES2,SN3,DES3,240100320V,240102289V,240102365V,240103778V,240104621V,240107245V,240107760V,240107922V,240108299V,240108828V,240109272V
3393,*1,S009231885,Circuit 0048 COFLRYB 0.35 BR/W,(blank),(blank),(blank),(blank),,,,,,,,,,,1
3392,*1,S009231885,Circuit 0048 COFLRYB 0.35 BR/W,(blank),(blank),(blank),(blank),,,,1,,,,,,,
3391,*1,S009231885,Circuit 0048 COFLRYB 0.35 BR/W,(blank),(blank),(blank),(blank),,,,,,1,,,,1,
3390,*1,S009231885,Circuit 0048 COFLRYB 0.35 BR/W,(blank),(blank),(blank),(blank),1,,,,,,,,,,
3389,*1,S009231885,Circuit 0048 COFLRYB 0.35 BR/W,(blank),(blank),(blank),(blank),,,,,1,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,***3,S009232047,Super Group 007 See drawing,S008822089,Twisted Wire 22 ZZ,S008821646,Circuit 0574 COFLRYB 0.35 BR/W,1,1,1,1,1,1,1,1,1,1,1
41,***3,S009232047,Super Group 007 See drawing,S009231960,Twisted Wire 19 ZZ,S008821647,Circuit 0565 COFLRYB 0.35 BR/W,1,1,1,1,1,1,1,1,1,1,1
42,***3,S009232047,Super Group 007 See drawing,S009231960,Twisted Wire 19 ZZ,S009231678,Circuit 0566 COFLRYB 0.35 G/W,1,1,1,1,1,1,1,1,1,1,1
17,***3,S009232045,Super Group 003 See drawing,S009231955,Joint 0655 See drawing,S009231662,Circuit 0085 COFLRYB 0.75 B/BR,,,,1,,,,1,1,,


In [ ]:
# Sep_sheet = wb.sheets.add(after=sheet_mmsta)
# Sep_sheet.name = "Sep Sheet"
# Sep_sheet.range("A1").value = [df_mmsta_pivot.columns.tolist()] + df_mmsta_pivot.values.tolist()

In [27]:
df_mmsta_pivot[df_mmsta_pivot.columns[7:]] = df_mmsta_pivot[df_mmsta_pivot.columns[7:]].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

In [28]:
columns_to_agg = list(df_mmsta_pivot.columns[7:])
cols = ['Level', 'SN1', 'DES1', 'SN2', 'DES2', 'SN3', 'DES3']
cols.extend(columns_to_agg)
cols

['Level',
 'SN1',
 'DES1',
 'SN2',
 'DES2',
 'SN3',
 'DES3',
 '240100320V',
 '240102289V',
 '240102365V',
 '240103778V',
 '240104621V',
 '240107245V',
 '240107760V',
 '240107922V',
 '240108299V',
 '240108828V',
 '240109272V']

In [29]:


agg_dict = {col: 'sum' for col in columns_to_agg}

df_mmsta_pivot = df_mmsta_pivot.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

df_mmsta_pivot = df_mmsta_pivot[cols]
df_mmsta_pivot.replace(to_replace=" " , value="(Blank)")
df_mmsta_pivot.replace(to_replace= 0 , value=" ")


sheet_mmsta.range("A1").value = [df_mmsta_pivot.columns.tolist()] + df_mmsta_pivot.values.tolist()

In [ ]:
df_fil_simple = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'circuit' in x.lower())]
columns_to_agg = list(df_fil_simple.columns[7:])

agg_dict = {col: 'sum' for col in columns_to_agg}

df_fil_simple = df_fil_simple.groupby(['Level','SN1']).agg({
    'DES1': 'first',
    'SN2': 'first',
    'DES2': 'first',
    'SN3': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

df_fil_simple.replace(to_replace=" " , value="(Blank)")
df_fil_simple.replace(to_replace= 0 , value=" ")

double_sheet = wb.sheets.add(after=sheet_mmsta)
sheet_mmsta.name = "Fil Simple"
sheet_mmsta.range("A1").value = [df_fil_simple.columns.tolist()] + df_fil_simple.values.tolist()

In [ ]:
df_double = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'double' in x.lower())]
df_double = df_double.groupby(['Level','SN1']).agg({
    'DES1': 'first',
    'SN2': 'first',
    'DES2': 'first',
    'SN3': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

double_sheet = wb.sheets.add(after=sheet_mmsta)
double_sheet.name = "Double"
double_sheet.range("A1").value = [df_double.columns.tolist()] + df_double.values.tolist()

In [ ]:
df_twisted = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'twisted' in x.lower())]
df_twisted = df_twisted.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
twisted_sheet = wb.sheets.add(after=double_sheet)
twisted_sheet.name = "Twisted"
twisted_sheet.range("A1").value = [df_twisted.columns.tolist()] + df_twisted.values.tolist()

In [ ]:
df_joint = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'joint' in x.lower())]
df_joint = df_joint.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
joint_sheet = wb.sheets.add(after=twisted_sheet)
joint_sheet.name = "joint"
joint_sheet.range("A1").value = [df_joint.columns.tolist()] + df_joint.values.tolist()

In [ ]:
df_super_grp = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'super group' in x.lower())]
df_super_grp = df_super_grp.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
superGrp_sheet = wb.sheets.add(after=joint_sheet)
superGrp_sheet.name = "Super group"
superGrp_sheet.range("A1").value = [df_super_grp.columns.tolist()] + df_super_grp.values.tolist()

In [ ]:
end_time = time.time()

execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time / 60} minutes")